# TF-IDF from Scratch (BoW / TF / IDF 数学本质)

**Day 3 目标：** 能手写 TF-IDF 公式；能解释 IDF 为什么有用；能手写实现（不用 sklearn）。

> 建议你把这份 notebook 里的关键公式、实现步骤自己再手敲一遍（面试就是要你能当场写出来）。


## 0. 你需要掌握的两句话
1) **BoW 假设：** 文档只由词的出现次数/权重表示，忽略词序（语法/局部上下文）。
2) **TF-IDF：** 用 **TF** 表示“这个词在当前文档的重要性”，用 **IDF** 抑制“全语料到处都是的词”。


## 1. BoW 数学形式
设词表大小为 $|V|$，文档 $d$ 的 BoW 向量 $\mathbf{x}^{(d)} \in \mathbb{R}^{|V|}$：

- 最朴素：$x^{(d)}_t = \text{count}(t, d)$
- 关键特性：**高维 + 稀疏**（大部分维度为 0）

**面试常问：** 为什么稀疏？因为每个文档只覆盖词表里极少一部分词。


## 2. TF 的几种常见定义
设词 $t$ 在文档 $d$ 中出现次数为 $f(t,d)$，文档总词数为 $|d|$。

**(A) Raw count：**
$$\mathrm{tf}(t,d)=f(t,d)$$
表达的是最原始的假设：出现越多越重要

存在的问题：

1.长文档天生占便宜：同一个主题，长文档里几乎所有的词count都大。长文档会因为“数值更大”被偏爱，而不是因为”更相关“

2.重复刷词会线性变强：比如一篇文档里”npl“出现了100次，Raw count会让它比重复10次强10倍，但现实中相关性通常没这么线性。

**(B) Length-normalized：**
$$\mathrm{tf}(t,d)=\frac{f(t,d)}{|d|}$$
归一化把”绝对次数“变成了”相对频率“，也就是这个词在这篇文档中占了多大比例。

例子：文档1：长度100，npl出现5次 -> tf=0.05 ； 文档2：长度10，npl出现2次 -> tf=0.2

Raw count 会说文档1更重要（5 > 2），但频率告诉你文档2更“集中讨论”nlp。

副作用：

对特别短的文档，频率会跳得特别大

只解决了”长短“问题，没有解决”刷词线性增长问题“

**(C) Sublinear（减弱重复带来的线性增长）：**
$$\mathrm{tf}(t,d)=1+\log f(t,d) \quad (f>0),\;0\;\text{otherwise}$$
取log是在修复词偏差性，你希望出现次数从1 → 2有明显增益，但从50 → 51增益应该很小。log正好是边际收益递减。

例子：

count：1 -> 2 是翻倍

log tf：(1+log1)=1 -> (1+log2)≈1.693，增益约为0.693

count：10 -> 100 是10倍

log tf: (1+log10)≈3.303 -> (1+log100)≈5.605，增益约为2.302

为什么要加“1”？避免 f=1 时 log(1)=0 让 tf 变成 0（失去“出现过”的信号）。

你在实现里可以任选一种，但要能解释“为什么要归一化/对数”。


## 3. IDF 推导直觉（重点：为什么有用）
设语料有 $N$ 篇文档，词 $t$ 出现过的文档数为 $\mathrm{df}(t)$。

为什么用 df(t)（文档频次）而不是词频总和？

因为我们关心的是：这个词能不能区分文档。
区分能力取决于“覆盖了多少文档”，而不是“在少数文档里刷了多少次”。

**想要的性质：**
- 词在越多文档出现（df 大），它越“不区分”文档 → 权重应该越小。
- 词越稀有（df 小），它越“区分”文档 → 权重应该越大。

满足“反比”直觉的一种形式是 $\frac{N}{\mathrm{df}(t)}$，再取对数压缩动态范围：
$$\mathrm{idf}(t)=\log\frac{N}{\mathrm{df}(t)}$$
这个公式是怎么来的？我们想要：

df 大 → 权重小

df 小 → 权重大

最简单满足单调性的就是反比：weight(t)∝1/df(t)

再乘上N做尺度化（让它跟语料规模有关系）：N/df(t)

取log是为了:

(1) 动态范围太夸张，少量稀有词会“爆表”，例如：N=1000000

df=1, N/df = 1000000; df=10000, N/df=100. 差了10000倍，模型被极少数词强行主宰。

使用log：log(1000000)≈13.8; log(10000)≈4.6. 差值变得可控

（2）“边际收益递减”更合理

df 从 1 增加到 2（变得更常见一倍）对区分度影响很大；df 从 100001 增加到 100002 基本没区别。



**平滑（避免 df=0，且让常见词也有稳定数值）** 常见写法：
$$\mathrm{idf}(t)=\log\frac{N+1}{\mathrm{df}(t)+1}+1$$
为什么分子分母都加“1”？

1.避免 df=0 时除零：如果模型在推理的时候遇到了在训练时没见过的词，即df=0，这时不平滑会直接爆炸（idf=∞）

2.让数值更稳定：即使 df 很小，也不会无限大到离谱。

为什么最后还要加“1”？

保证 IDF 非负，并且让 df=N 的词不至于变成 0。全出现的词 idf=1，而不是 0。idf=0，意味着我们要永远丢弃这些词，这在很多场景是可以的（停用词），但库一般不替你做“强硬裁剪”，而是留一点权重，再让下游（比如模型/分类器/相似度）去决定；同时也避免了某些数值/实现里出现全 0 特征导致的边界问题。

这也是很多库（如 scikit-learn）常用的平滑变体之一。


## 4. TF-IDF 公式（你必须能手写）
$$\mathrm{tfidf}(t,d) = \mathrm{tf}(t,d)\cdot \mathrm{idf}(t)$$
（重要）TF-IDF 向量是什么？（从“单词权重”到“文档表示”）

向量空间模型（Vector Space Model）的核心思想是：把一篇文档表示成一个向量，而向量的每一维，对应词表里的一个词

假设整个语料库的词表是V={t1, t2, t3,...,tv}. 那么向量的第j维就是tj。

对于文档d，我们构造一个向量d = {tfidf(t1,d), tfidf(t2,d), tfidf(t3,d),...,tfidf(tv,d)}. 在文档向量中，第j个分词的数值，等于词tj在在文档d中的tf-idf权重。

例子： 
    t1 = "猫"；
    t2 = "狗"；
    t3 = "鱼"；

在某篇文档中：
    tfidf("猫", d) = 0.3；
    tfidf("狗", d) = 0；
    tfidf("鱼", d) = 0.8；

向量d = (0.3,0,0.8)

可选的向量归一化（面试常问）：对每个文档向量做 L2 normalize：
$$\hat{\mathbf{x}}^{(d)}=\frac{\mathbf{x}^{(d)}}{\left\|\mathbf{x}^{(d)}\right\|_{2}}, \quad\|\mathbf{x}\|_{2}=\sqrt{\sum_{j} x_{j}^{2}}$$
L2 normalize解决的问题：

向量“尺度”（整体大小）会影响相似度。假如使用点积做相似度计算，Xa x Xb，向量越大（词更多/TF 总量更大），点积往往越大 → 长文档/词多文档更占便宜。

经过L2 normalize之后每个文档的向量长度都变成了1，这样比较的时候就不会因为谁向量大就偏向谁了。

L2 normalize 和 cosine 相似度的关系

cosine相似度：$\cos (\theta)=\frac{\mathbf{x} \cdot \mathbf{y}}{\|\mathbf{x}\|_{2}\|\mathbf{y}\|_{2}}$

如果你先把两边都 L2 normalize：$\hat{\mathbf{x}}=\frac{\mathbf{x}}{\|\mathbf{x}\|_{2}}, \hat{\mathbf{y}}=\frac{\mathbf{y}}{\|\mathbf{y}\|_{2}}$

那么就会得到：$\hat{\mathbf{x}} \cdot \hat{\mathbf{y}}=\cos (\theta)$

所以结论就是：对 TF-IDF 向量做 L2 normalize + 点积” 等价于 “直接做 cosine 相似度

normalize 把长度因素去掉，让你更关注“词权重分布结构”，这对检索/聚类通常更合理，因为长度不等于相关性。

## 5. 代码任务：手写 TF-IDF（不用 sklearn）
实现目标：输入 `List[List[str]]`（已分好词的语料），输出：
- `vocab: Dict[str,int]` 词到索引
- `idf: List[float]` 或 `np.ndarray`
- `X_tfidf: np.ndarray` (N, |V|) 稀疏可以先用 dense（面试写起来更快），但要能解释如何用稀疏矩阵优化。


In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Iterable, Optional
import math
import numpy as np


### 5.1 一个极小语料（先跑通，再泛化）
你也可以换成自己的语料，但建议先用这个做 sanity check。


In [2]:
corpus = [
    ["i", "love", "nlp", "nlp"],
    ["i", "love", "deep", "learning"],
    ["nlp", "is", "fun"],
]


### 5.2 你要实现的最小 API
- `build_vocab(corpus)`
- `compute_df(corpus, vocab)`
- `compute_idf(df, N)`
- `compute_tfidf(corpus, vocab, idf, tf_variant='log')`
- 可选：`l2_normalize_rows(X)`


In [ ]:
def build_vocab(corpus: List[List[str]]) -> Dict[str, int]:
    """Build vocabulary mapping token -> index."""
    vocab: Dict[str, int] = {}
    for doc in corpus:
        for tok in doc:
            if tok not in vocab:
                vocab[tok] = len(vocab)
    return vocab

def compute_df(corpus: List[List[str]], vocab: Dict[str, int]) -> np.ndarray:
    """Document frequency: df[t] = number of documents containing token t."""
    V = len(vocab)
    df = np.zeros(V, dtype=np.int64)
    for doc in corpus:
        seen = set(doc)  # unique tokens in this doc
        for tok in seen:
            idx = vocab.get(tok)
            if idx is not None:
                df[idx] += 1
    return df

def compute_idf(df: np.ndarray, N: int, smooth: bool = True) -> np.ndarray:
    """Compute IDF vector.
    smooth=True uses: idf = log((N+1)/(df+1)) + 1
    """
    df = df.astype(np.float64)
    if smooth:
        return np.log((N + 1.0) / (df + 1.0)) + 1.0
    else:
        # NOTE: if df==0, this will be inf; in practice, vocab comes from corpus so df>0.
        return np.log(N / df)

def _tf_value(count: int, doc_len: int, variant: str) -> float:
    """Compute TF for a single token in a doc."""
    if count <= 0:
        return 0.0
    if variant == "raw":
        return float(count)
    if variant == "freq":
        return float(count) / float(doc_len)
    if variant == "log":
        return 1.0 + math.log(float(count))
    raise ValueError(f"Unknown tf variant: {variant}")

def compute_tfidf(
    corpus: List[List[str]],
    vocab: Dict[str, int],
    idf: np.ndarray,
    tf_variant: str = "log",
) -> np.ndarray:
    """Return dense TF-IDF matrix X with shape (N, V)."""
    N = len(corpus)
    V = len(vocab)
    X = np.zeros((N, V), dtype=np.float64)

    for i, doc in enumerate(corpus):
        doc_len = len(doc)
        # term counts for this doc
        counts: Dict[int, int] = {}
        for tok in doc:
            j = vocab.get(tok)
            if j is None:
                continue
            counts[j] = counts.get(j, 0) + 1

        for j, c in counts.items():
            tf = _tf_value(c, doc_len, tf_variant)
            X[i, j] = tf * float(idf[j])

    return X

def l2_normalize_rows(X: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    """Row-wise L2 normalization."""
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    return X / (norms + eps)


### 5.3 跑一遍：输出 vocab / df / idf / tfidf
你要能手算一个小例子：比如词 `nlp` 在第 1 篇文档 tfidf 是多少？


In [4]:
vocab = build_vocab(corpus)
df = compute_df(corpus, vocab)
idf = compute_idf(df, N=len(corpus), smooth=True)
X = compute_tfidf(corpus, vocab, idf, tf_variant="log")
X_norm = l2_normalize_rows(X)

vocab, df, idf, X, X_norm


({'i': 0, 'love': 1, 'nlp': 2, 'deep': 3, 'learning': 4, 'is': 5, 'fun': 6},
 array([2, 2, 2, 1, 1, 1, 1], dtype=int64),
 array([1.28768207, 1.28768207, 1.28768207, 1.69314718, 1.69314718,
        1.69314718, 1.69314718]),
 array([[1.28768207, 1.28768207, 2.18023527, 0.        , 0.        ,
         0.        , 0.        ],
        [1.28768207, 1.28768207, 0.        , 1.69314718, 1.69314718,
         0.        , 0.        ],
        [0.        , 0.        , 1.28768207, 0.        , 0.        ,
         1.69314718, 1.69314718]]),
 array([[0.45329466, 0.45329466, 0.76749457, 0.        , 0.        ,
         0.        , 0.        ],
        [0.42804604, 0.42804604, 0.        , 0.5628291 , 0.5628291 ,
         0.        , 0.        ],
        [0.        , 0.        , 0.4736296 , 0.        , 0.        ,
         0.62276601, 0.62276601]]))

### 5.4 让输出更可读：按词表顺序打印
面试点：你要能解释某些词权重为什么变大/变小。


In [5]:
# Pretty print
inv_vocab = {i: t for t, i in vocab.items()}
terms = [inv_vocab[i] for i in range(len(inv_vocab))]

print("Vocab order:", terms)
print("df:", df.tolist())
print("idf:", np.round(idf, 4).tolist())
print("\nTF-IDF (dense):")
print(np.round(X, 4))
print("\nTF-IDF (L2-normalized):")
print(np.round(X_norm, 4))


Vocab order: ['i', 'love', 'nlp', 'deep', 'learning', 'is', 'fun']
df: [2, 2, 2, 1, 1, 1, 1]
idf: [1.2877, 1.2877, 1.2877, 1.6931, 1.6931, 1.6931, 1.6931]

TF-IDF (dense):
[[1.2877 1.2877 2.1802 0.     0.     0.     0.    ]
 [1.2877 1.2877 0.     1.6931 1.6931 0.     0.    ]
 [0.     0.     1.2877 0.     0.     1.6931 1.6931]]

TF-IDF (L2-normalized):
[[0.4533 0.4533 0.7675 0.     0.     0.     0.    ]
 [0.428  0.428  0.     0.5628 0.5628 0.     0.    ]
 [0.     0.     0.4736 0.     0.     0.6228 0.6228]]


## 6. 你必须能回答的 3 个面试追问（写下你的答案）
1) **IDF 为什么有用？**（提示：区分性、信息量、抑制停用词）

2) **为什么要对数/归一化 TF？**（提示：重复词、文档长度偏置）

3) **稀疏向量的工程影响？**（提示：内存、矩阵乘法、倒排索引/稀疏矩阵）



1.IDF 用来衡量一个词在全语料层面的区分能力。
如果一个词在几乎所有文档中都出现（df 很大），它携带的信息量很低，无法区分文档，应当被压低权重；反之，只在少数文档中出现的词更具判别性。
因此 IDF 通过对高 df 词施加惩罚，抑制停用词的干扰，放大“主题词”的影响，使 TF-IDF 更关注能区分文档的特征。

2.Raw TF 会带来两个问题：长文档天然数值更大，以及词频重复会线性放大权重。
Length-normalized TF 通过除以文档长度，消除了文档长短带来的偏置，使权重反映“相对重要性”而非“绝对次数”。
Log TF 则引入边际收益递减，避免某个词通过大量重复主宰向量，但仍保留“出现过 vs 未出现”的关键信号。

3.TF-IDF 向量维度等于词表大小，而单篇文档只包含极少数词，因此向量高度稀疏。
工程上如果用 dense 表示，会造成巨大内存浪费和无效计算，实际系统通常采用稀疏矩阵或倒排索引。
这种表示可以显著降低存储成本，并加速相似度计算和检索，是搜索引擎和文本检索系统的基础。

## 7. 加餐：你会怎么处理 OOV？
- 做法 A：忽略 OOV（线上检索/匹配时可能还行）
- 做法 B：用 `<UNK>` 统一映射
- 做法 C：子词/字符级（到后面 wordpiece/BPE 再系统讲）

写出你更倾向哪种，以及理由。


优先选择做法 C（子词 / 字符级建模），必要时结合 B 作为兜底。

理由:

从根本上解决 OOV 问题.
子词（BPE / WordPiece）可以把任何新词拆解，理论上几乎不存在 OOV。

信息损失最小，泛化能力更强.
相比 <UNK> 把所有未知词混为一类，子词保留了词形和语义组合信息，模型能对未见词做合理推断。

特别适合中文和专业领域.
中文新词多、组合灵活，子词/字符级比词级更鲁棒。

工业界与主流模型的事实标准.
BERT、GPT 等主流模型都采用子词方案，工程上成熟可靠。

A（忽略 OOV）：只适合检索或规则系统，不适合作为模型建模方案

B：实现简单，但语义信息损失大，只适合作为兜底

In [ ]:
import math
import numpy as np

corpus = [
    ["i", "love", "nlp", "nlp"],
    ["i", "love", "deep", "learning"],
    ["nlp", "is", "fun"],
]

#list[list[str]] -> dict[str,int]
def build_vocab(corpus):
    """build vocabulary making token -> index"""
    vocab = {}
    for doc in corpus:
        for tok in doc:
            if tok not in vocab:
                vocab[tok] = len(vocab)
    
    return vocab

def compute_df(corpus, vocab):
    """document frequency: df[t] = number of documents containing token t"""
    V = len(vocab)
    df = np.zeros(V, dtype=np.int64)
    for doc in corpus:
        seen = set(doc)
        for tok in seen:
            idx = vocab.get(tok)
            if idx is not None:
                df[idx] += 1
    
    return df

def compute_idf(df, N, smooth=True):
    """
    compute idf
    smooth = True
    idf = log((N+1) / (df+1)) + 1
    """
    df = df.astype(np.float64)
    if smooth:
        return np.log((N + 1.0) / (df + 1.0)) + 1
    else:
        #note: if df=0, this will be inf; in practice, vocab comes from corpus so df > 0.
        return np.log(N / df)
    
def _tf_value(count, doc_len, variant):
    """compute TF in a single token in a doc."""
    if count < 0:
        return 0.0
    if variant == "raw":
        return count
    if variant == "freq":
        return float(count) / float(doc_len)
    if variant == "log":
        return np.log(float(count)) + 1.0
    raise ValueError(f"unknown TF variant: {variant}")

def compute_tfidf(corpus, vocab, idf, tf_variant="log"):
    """return dense tf-idf matrix X with shape (N, V)"""
    N = len(corpus)
    V = len(vocab)
    X = np.zeros((N, V), dtype=np.float64)

    for i, doc in enumerate(corpus):
        doc_len = len(doc)
        counts = {}
        for tok in doc:
            j = vocab.get(tok)
            if j is None:
                continue
            counts[j] = counts.get(j, 0) + 1

        for j, c in counts.items():
            tf = _tf_value(c, doc_len, tf_variant)
            X[i, j] = tf * float(idf[j])

    return X

def l2_normalize_rows(X, eps=1e-12):
    """Row-wise L2 normalization"""
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    return X / (norms + eps)


vocab = build_vocab(corpus)
df = compute_df(corpus, vocab)
idf = compute_idf(df, N=len(corpus), smooth=True)
X = compute_tfidf(corpus, vocab, idf, tf_variant="log")
X_norm = l2_normalize_rows(X)

print(vocab, df, idf, X, X_norm)
print("\n\n")


#Pretty print
inv_vocab = {i: t for t, i in vocab.items()}
terms = [inv_vocab[i] for i in range(len(inv_vocab))]

print("Vocab order:", terms)
print("df:", df.tolist())
print("idf:", np.round(idf, 4).tolist())
print("\nTF-IDF (dense):")
print(np.round(X, 4))
print("\nTF-IDF (L2-normalized):")
print(np.round(X_norm, 4))

{'i': 0, 'love': 1, 'nlp': 2, 'deep': 3, 'learning': 4, 'is': 5, 'fun': 6} [2 2 2 1 1 1 1] [1.28768207 1.28768207 1.28768207 1.69314718 1.69314718 1.69314718
 1.69314718] [[1.28768207 1.28768207 2.18023527 0.         0.         0.
  0.        ]
 [1.28768207 1.28768207 0.         1.69314718 1.69314718 0.
  0.        ]
 [0.         0.         1.28768207 0.         0.         1.69314718
  1.69314718]] [[0.45329466 0.45329466 0.76749457 0.         0.         0.
  0.        ]
 [0.42804604 0.42804604 0.         0.5628291  0.5628291  0.
  0.        ]
 [0.         0.         0.4736296  0.         0.         0.62276601
  0.62276601]]



Vocab order: ['i', 'love', 'nlp', 'deep', 'learning', 'is', 'fun']
df: [2, 2, 2, 1, 1, 1, 1]
idf: [1.2877, 1.2877, 1.2877, 1.6931, 1.6931, 1.6931, 1.6931]

TF-IDF (dense):
[[1.2877 1.2877 2.1802 0.     0.     0.     0.    ]
 [1.2877 1.2877 0.     1.6931 1.6931 0.     0.    ]
 [0.     0.     1.2877 0.     0.     1.6931 1.6931]]

TF-IDF (L2-normalized):
[[0.4533 0